In [49]:
resdir = '/Users/Serena/Desktop/PA-brain-project/results'

results_format = 'QDECR' # 'verywise'

In [54]:
top_level = [f for f in os.listdir(resdir) if not f.startswith('.')]
top_level

['light_pa', 'self_report_pa', 'tot_steps', 'mvpa']

In [55]:
os.listdir(f'{resdir}/{p}')

['rh.light_pa_adjint.thickness',
 'lh.light_pa.area',
 '.DS_Store',
 'rh.light_pa.thickness',
 'lh.light_pa_adjint.area',
 'lh.light_pa.thickness',
 'rh.light_pa.area',
 'rh.light_pa_adjint.area']

In [61]:
p = top_level[0]  # Assuming you want to work with the first top-level directory
subdir_list = [f for f in os.listdir(os.path.join(resdir, p)) if (
                os.path.isdir(os.path.join(resdir, p, f))) & (not f.startswith('.'))]

In [68]:
subdir = subdir_list[0]
subdir

res = pd.DataFrame([d.split('.') for d in subdir_list], 
                   columns=['hemi','model','meas'])

In [69]:
res

,hemi,model,meas
0,rh,light_pa_adjint,thickness
1,lh,light_pa,area
2,rh,light_pa,thickness
3,lh,light_pa_adjint,area
4,lh,light_pa,thickness
5,rh,light_pa,area
6,rh,light_pa_adjint,area


In [ ]:



if len(subdir_list) == 0: # already found the deepest level
res = pd.DataFrame([d.split('.')[:2] for d in os.listdir(f'{resdir}/{p}') if d.endswith('.mgh')],
                            columns=['hemi', 'meas']).drop_duplicates()
res['model'] = p
all_results[p] = res

else:
subdir_res = pd.DataFrame()
for subdir in subdir_list:
    res = pd.DataFrame([d.split('.') for d in os.listdir(f'{resdir}/{p}/{subdir}') if d.endswith('.mgh')],
                        columns=['hemi', 'meas']).drop_duplicates()
    res['model'] = subdir
    subdir_res = pd.concat([subdir_res, res])

all_results[p] = subdir_res

In [41]:
import re 
def download_github_folder(github_url, github_token=None, GITHUB_FOLDER_CACHE={}):
    """
    Downloads a folder from a public GitHub repo to a temp directory.
    github_url: e.g. https://github.com/user/repo/tree/main/path/to/folder
    Returns the local path to the downloaded folder.
    """
    import tempfile
    import requests

    # Check cache first
    if github_url in GITHUB_FOLDER_CACHE:
        return GITHUB_FOLDER_CACHE[github_url]
    
    m = re.match(r"https://github.com/([^/]+)/([^/]+)/tree/([^/]+)/(.*)", github_url)
    if not m:
        raise ValueError("URL must be of the form https://github.com/user/repo/tree/branch/path/to/folder")
    user, repo, branch, folder_path = m.groups()

    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder_path}?ref={branch}"
    tmp_dir = tempfile.mkdtemp()
    folder_local = os.path.join(tmp_dir, os.path.basename(folder_path))
    os.makedirs(folder_local, exist_ok=True)

    headers = {}
    # Try to get token from environment if not provided
    if github_token is None:
        github_token = os.environ.get("GITHUB_TOKEN")
    if github_token:
        headers["Authorization"] = f"token {github_token}"

    def download_contents(api_url, local_dir, print_progress=True):
        resp = requests.get(api_url, headers=headers)
        resp.raise_for_status()
        element_count = len(resp.json())
        # with ui.Progress(min=0, max=element_count) as p:
        for e, file_info in enumerate(resp.json()):

            # if print_progress: 
            #     p.set(value=e + 1, message=f"Downloading [{e + 1}/{element_count}]: {file_info['name']}...")
            # else: 
            #     p.set(value=e + 1, message=f"[{e + 1}/{element_count}]: {file_info['name']}...")

            if file_info["type"] == "dir":
                subfolder_local = os.path.join(local_dir, file_info["name"])
                os.makedirs(subfolder_local, exist_ok=True)
                download_contents(file_info["url"], subfolder_local, print_progress=False)

            elif file_info["type"] == "file":
                file_resp = requests.get(file_info["download_url"])
                file_resp.raise_for_status()
                with open(os.path.join(local_dir, file_info["name"]), "wb") as f:
                    f.write(file_resp.content)
                

    download_contents(api_url, folder_local)

    # Store in cache
    GITHUB_FOLDER_CACHE[github_url] = folder_local

    return folder_local


def detect_models(resdir, results_format):

    resdir = resolve_resdir(resdir)

    # Make sure path is correctly specified
    resdir = f'{resdir}/' if resdir[-1] != '/' else resdir

    # List all results
    all_results = dict()

    top_level = [f for f in os.listdir(resdir) if not f.startswith('.')]

    print('FOLDERS', top_level)

    if results_format == 'verywise':
        for p in sorted(top_level):
            subdir_list = [f for f in os.listdir(f'{resdir}/{p}') if (os.path.isdir(f)) & (not f.startswith('.'))]
            if len(subdir_list) == 0: # already found the deepest level
                res = pd.DataFrame([d.split('.')[:2] for d in os.listdir(f'{resdir}/{p}') if d.endswith('.mgh')],
                                            columns=['hemi', 'meas']).drop_duplicates()
                res['model'] = p
                all_results[p] = res

            else:
                subdir_res = pd.DataFrame()
                for subdir in subdir_list:
                    res = pd.DataFrame([d.split('.')[:2] for d in os.listdir(f'{resdir}/{p}/{subdir}') if d.endswith('.mgh')],
                                        columns=['hemi', 'meas']).drop_duplicates()
                    res['model'] = subdir
                    subdir_res = pd.concat([subdir_res, res])
                
                all_results[p] = subdir_res
    
    elif results_format == 'QDECR': #TODO: adapt this to all QDECR formats
        for p in sorted(top_level):
            print(p)
            all_results[p] = pd.DataFrame([p.split('.')], columns=['hemi', 'model', 'meas'])

    return all_results



In [42]:
dir = download_github_folder(resdir)


In [45]:
os.listdir(f'{dir}/{os.listdir(dir)[0]}')

['rh.area.stack1.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack6.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack5.coef.mgh',
 'rh.area.stack4.coef.mgh',
 'rh.area.stack6.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack1.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack1.coef.mgh',
 'rh.area.stack7.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack7.coef.mgh',
 'lh.area.stack6.coef.mgh',
 'lh.area.stack7.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack8.coef.mgh',
 'rh.area.stack2.coef.mgh',
 'rh.area.stack3.coef.mgh',
 'lh.area.stack3.coef.mgh',
 'lh.area.stack2.coef.mgh',
 'rh.area.stack5.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack2.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack8.coef.mgh',
 'rh.area.stack2.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack6.coef.mgh',
 'rh.area.stack7.coef.mgh',
 'lh.area.stack5.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack8.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack3.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack4.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack1.coef.mgh',
 'stack_na

In [46]:
import pandas as pd
stacks = pd.read_table(f'{dir}/{os.listdir(dir)[0]}/stack_names.txt', delimiter="\t")


In [47]:
stacks


,stack_number,stack_name
0,1,(Intercept)
1,2,att
2,3,age
3,4,sexgirl
4,5,ethnother
5,6,par_eduMedium
6,7,par_eduHigh
7,8,age_m


['rh.area.stack1.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack6.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack5.coef.mgh',
 'rh.area.stack4.coef.mgh',
 'rh.area.stack6.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack1.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack1.coef.mgh',
 'rh.area.stack7.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack7.coef.mgh',
 'lh.area.stack6.coef.mgh',
 'lh.area.stack7.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack8.coef.mgh',
 'rh.area.stack2.coef.mgh',
 'rh.area.stack3.coef.mgh',
 'lh.area.stack3.coef.mgh',
 'lh.area.stack2.coef.mgh',
 'rh.area.stack5.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack2.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack8.coef.mgh',
 'rh.area.stack2.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack6.coef.mgh',
 'rh.area.stack7.coef.mgh',
 'lh.area.stack5.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack8.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack3.cache.th30.abs.sig.ocn.mgh',
 'lh.area.stack4.cache.th30.abs.sig.ocn.mgh',
 'rh.area.stack1.coef.mgh',
 'stack_na